# imports

In [21]:
from langchain_core.runnables import RunnableLambda
import random

from langchain_community.llms import Ollama
from langchain_community.chat_models import ChatOllama
from langchain_core.messages import (
    SystemMessage,
    AIMessage,
    HumanMessage
)
from langchain_core.prompts import PromptTemplate
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain_core.prompts import (
    ChatMessagePromptTemplate,
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder
)

# Runnable in Langchain

## Runnable Sequence constructed using "|" operator

In [2]:
sequence = RunnableLambda(lambda x: x + 1) | RunnableLambda(lambda x: x * 2)

# sequence.invoke(1)
sequence.batch([1, 2, 3])

[4, 6, 8]

## Runnable Parallel constructed using dict literal

In [3]:
sequence = RunnableLambda(lambda x: x + 1) | {
    "mul_2": RunnableLambda(lambda x: x * 2),
    "mul_4": RunnableLambda(lambda x: x * 4)
    }

sequence.invoke(1)

{'mul_2': 4, 'mul_4': 8}

## Runnable additional methods

In [4]:
def add(x: int) -> int:
    return x + 1

In [5]:
def buggy_double(y: int) -> int:
    if random.random() > 0.3:
        print("this code failed and will probably be retired!")
        raise ValueError("Triggered buggy code")
    return y * 2

In [6]:
# sequence = RunnableLambda(add) | RunnableLambda(buggy_double)

sequence = RunnableLambda(add) | RunnableLambda(buggy_double).with_retry(stop_after_attempt=10, wait_exponential_jitter=False)
sequence

RunnableLambda(add)
| RunnableRetry(bound=RunnableLambda(buggy_double), wait_exponential_jitter=False, max_attempt_number=10)

In [7]:
sequence.input_schema.schema()
# sequence.invoke(1)

{'title': 'add_input', 'type': 'integer'}

In [8]:
sequence.output_schema.schema()

{'title': 'buggy_double_output', 'type': 'integer'}

# langauge models in langchain

### llms: refers to pure text completion models

-   invoking llms with a plain string will return the results as a string

In [9]:
llm = Ollama(model="llama3.1:8b", temperature=0)

In [10]:
# response = llm.invoke("what would be a good company name for a company that makes colorful socks ?")

prompt = PromptTemplate.from_template("What is a good name for a company that makes {product} ?")
chain = prompt | llm
response = chain.invoke({"product": "colorful socks"})

print(f"type(response): {type(response)}")
# print(f"content: {response.content}")
print(response)

type(response): <class 'str'>
Here are some ideas:

1. **SockSavvy**: This name plays off the idea of being knowledgeable or "savvy" about socks, which could be appealing to customers.
2. **ColorCrew**: This name emphasizes the fun, colorful aspect of your company's products and implies a team effort in creating them.
3. **ToeTally Fun**: This name is playful and lighthearted, suggesting that your socks are not only colorful but also enjoyable to wear.
4. **SoleMates Socks**: This name has a fun double meaning, referencing both the sole of a sock and the idea of finding a perfect match (either for yourself or as a gift).
5. **Heelicious**: Similar to "ToeTally Fun," this name is playful and emphasizes the delicious aspect of colorful socks.
6. **StepUp Socks**: This name suggests that your company's products will help customers take their sock game to the next level.
7. **SockItToMe**: This name has a fun, casual vibe and implies that your socks are so good, you'll want to "sock it to 

### chat model: tuned specifically for having conversations

-   invoking chat models with a list of messages will return the results as a AIMessage list

In [11]:
ollama_chat_model = ChatOllama(model="llama3.1:8b", temperature=0)

In [12]:
messages = [
    SystemMessage(content="You are an AI assistant"),
    HumanMessage(content="what would be a good company name for a company that makes colorful socks ?")
    ]

response = ollama_chat_model.invoke(input=messages)

print(f"type(response): {type(response)}")
print(f"response: {response}")

print(f"response.content: {response.content}")


type(response): <class 'langchain_core.messages.ai.AIMessage'>
response: content='Here are some ideas for a company name that might fit the bill:\n\n1. **SoleMates**: A playful name that suggests the socks will become your best friends.\n2. **ColorCrew**: This name emphasizes the fun, colorful aspect of the socks and implies a team or crew behind them.\n3. **SockScene**: A catchy name that evokes the idea of a vibrant, dynamic scene - which is exactly what colorful socks can bring to an outfit!\n4. **ToeTally Fun**: A whimsical name that conveys the playful spirit of the company and its products.\n5. **HueHub**: This name highlights the variety of colors available in the socks and creates a sense of community or hub around them.\n6. **StepUp Socks**: A motivational name that encourages customers to step up their style game with colorful, fun socks.\n7. **SockSavvy**: This name positions the company as experts in all things sock-related - which can be appealing to customers looking for 

# prompt templates

In [13]:
prompt = PromptTemplate.from_template("What is a good name for a company that makes {product} ?")
result = prompt.format(product= "colorful socks")

print(f"type(result): {type(result)}")
print(f"result: {result}")

type(result): <class 'str'>
result: What is a good name for a company that makes colorful socks ?


In [14]:
chat_prompt_template = ChatPromptTemplate.from_messages([
    ("system", "you are a helpful assistant that translates {input_language} to {output_language}."),
    ("human", "{text}")
])


# print(type(chat_prompt_template))
# chat_prompt_template.format_messages(input_language="English", output_language="French", text="I love programming.")

result = chat_prompt_template.invoke({"input_language": "English", "output_language": "French", "text": "I love programming."})

print(f"type(result): {type(result)}")
print(f"result: {result}")

type(result): <class 'langchain_core.prompt_values.ChatPromptValue'>
result: messages=[SystemMessage(content='you are a helpful assistant that translates English to French.'), HumanMessage(content='I love programming.')]


# output parsers

In [15]:
output_parser = CommaSeparatedListOutputParser()
output_parser.parse("hi, bye")

['hi', 'bye']

# composing with LCEL: lang chain expression language

In [ ]:
chat_model = ChatOllama(model="llama3.1:8b", temperature=0)
output_parser = CommaSeparatedListOutputParser()
prompt = ChatPromptTemplate.from_template("Generate a list of 5 {text}.\n\n{format_instructions}")
# prompt = ChatPromptTemplate.from_messages([
#     ("system", "you are a helpful assistant."),
#     ("human", "Generate a list of 5 {text}. \n\n {format_instructions}")

# ])

prompt = prompt.partial(format_instructions=output_parser.get_format_instructions())

chain = prompt | chat_model | output_parser

chain.invoke({"text": "colors"})

OllamaEndpointNotFoundError: Ollama call failed with status code 404. Maybe your model is not found and you should pull the model with `ollama pull llama3:8b`.

# Prompts Template

## String PromptTemplates
-   used to create a template from a string prompt.

In [ ]:
prompt_template = PromptTemplate.from_template("tell me a {adjective} joke about {content}.")
prompt_template.format(adjective="funny", content="chickens")
# result = prompt_template.invoke({"adjective": "sad", "content": "chickens"})
# print(type(result))
# print(result)

## Chat Prompt Template
-   The prompt to chat models/ is a list of chat messages.
-   Each chat message is associated with content, and an additional parameter called role.
-   For example, in the OpenAI Chat Completions API, a chat message can be associated with an AI assistant, a human or a system role.
-   The ChatPromptTemplate.from_messages static method accepts a variety of message representations and is a convenient way to format input to chat models with exactly the messages you want.

In [ ]:
chat_prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "you are a helpful AI bot. Your name is {name}"),
        ("human", "Hello, how are you doing?."),
        ("ai", "I'm doing well, thanks!"),
        ("human", "{user_input}")
    ]
)

messages = chat_prompt_template.format_messages(name="Jarvis", user_input="what is your name?")
# messages = chat_prompt_template.invoke({"name": "Jarvis", "user_input": "what is your name?"})

print(f"type(messages): {type(messages)}")
messages

In [ ]:
chat_prompt_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content="You are a helpful assistant that re-writes the user's text to sound it more upbeat"),
        HumanMessagePromptTemplate.from_template("{text}")
    ]
)

messages = chat_prompt_template.invoke({"text": "what is your name?"})
messages

## Message Prompts
- used where the chat model supports taking chat message with arbitrary role.

In [ ]:
chat_message_prompt_template = ChatMessagePromptTemplate.from_template(role="Jedi", template="May the {subject} be with you")
chat_message_prompt_template.format(subject="force")

## Message Placeholder
- gives full control of what messages to be rendered during formatting.
- used when uncertain of what role should be used and wish to insert a list of messages during formatting.


In [ ]:
human_message_template = HumanMessagePromptTemplate.from_template("Summarize our conversation so far in {word_count} words.")

chat_prompt_template = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="conversation"), 
    human_message_template
    ])

human_message = HumanMessage(content="What is the best way to learn programming?")
ai_message = AIMessage(content="""
1. Choose a programming language: Decide on a programming language that you want to learn.

2. Start with the basics: Familiarize yourself with the basic programming concepts such as variables, data types and control structures.

3. Practice, practice, practice: The best way to learn programming is through hands-on experience\
""" 
)

response = chat_prompt_template.format_prompt(
    conversation=[human_message, ai_message],
    word_count="10"
).to_messages()

print(f"{type(response)}")
response

In [ ]:
# message placeholders

chat_prompt_template = ChatPromptTemplate.from_messages([
    ("system", "you are a helpful assistant"),
    MessagesPlaceholder("msgs")
    # ("placeholder", "{msgs}")
])

input = chat_prompt_template.invoke({"msgs": [HumanMessage(content="Hello there!")]})

input

# Selectors